In [1]:
import os
from dotenv import load_dotenv
from office365.sharepoint.client_context import ClientContext

load_dotenv()

SITE_URL   = os.getenv("SP_SITE_URL")            # e.g. https://<tenant>.sharepoint.com/sites/ai4km
TENANT     = os.getenv("SP_TENANT")              # e.g. yourtenant.onmicrosoft.com (or the tenant GUID)
CLIENT_ID  = os.getenv("SP_CLIENT_ID")
THUMBPRINT = os.getenv("SP_CERT_THUMBPRINT")     # uppercase hex, no colons

os.chdir('../')                                  # When this notebook is launched, the working directory is at root/WebApp_ChatBot/test_notebooks/
                                                 # Therefore, need to go up 1 directory level to ./WebApp_Chatbot/ so that SP_CERT_PEM_PATH can be ./sharepoint_certificates/privkey.pem where ./ here is relative path wrt root/WebApp_ChatBot/
CERT_PEM   = os.getenv("SP_CERT_PEM_PATH")       

ctx = ClientContext(SITE_URL).with_client_certificate(
    tenant=TENANT,
    client_id=CLIENT_ID,
    thumbprint=THUMBPRINT,
    cert_path=CERT_PEM
)

web = ctx.web
ctx.load(web)
ctx.execute_query()
print("✅ Site Title:", web.properties.get("Title"))


/Users/jameslim/miniconda3/envs/ai4km/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


✅ Site Title: ai4km


In [2]:
from office365.sharepoint.client_context import ClientContext
import uuid

# reuse your existing ctx
lists = ctx.web.lists
ctx.load(lists)
ctx.execute_query()
print("📋 Lists:")
for l in lists:
    print("  -", l.properties.get("Title"))

# 1) Ensure your target list exists (exact title)
LIST_TITLE = "AI4KM Knowledge Assets"
target = ctx.web.lists.get_by_title(LIST_TITLE)
ctx.load(target)
ctx.execute_query()
print("✅ Found list:", target.properties.get("Title"))

# 2) Add a minimal test item (uses the fields you plan to write)
record_id = str(uuid.uuid4())
item_props = {
    "Title": "AUTH TEST ITEM 456",
    "ContentSummary": "Connectivity OK.",
    "Benefits": "N/A",
    "ContentOwner": "bot@amgen.com",
    "Function": "Digital Technology & Innovation",
    "Site": "ASM",
    "RecordId": record_id,
}
item = target.add_item(item_props)
ctx.execute_query()
print("🟢 Wrote item with RecordId:", record_id)

# 3) Read it back by ID
ctx.load(item)
ctx.execute_query()
print("🔎 Created Item ID:", item.properties.get("Id"))


📋 Lists:
  - AI4KM Knowledge Assets
  - appdata
  - appfiles
  - Composed Looks
  - Converted Forms
  - Documents
  - Form Templates
  - List Template Gallery
  - Master Page Gallery
  - Site Assets
  - Site Pages
  - Solution Gallery
  - Style Library
  - TaxonomyHiddenList
  - Theme Gallery
  - User Information List
  - Web Part Gallery
  - Web Template Extensions
✅ Found list: AI4KM Knowledge Assets
🟢 Wrote item with RecordId: 78db7d8e-b590-47d1-871a-2863f53248be
🔎 Created Item ID: 8


In [3]:
fields = target.fields
ctx.load(fields)
ctx.execute_query()
print("🧩 Internal field names:")
for f in fields:
    print(f.properties.get("InternalName"), ":", f.properties.get("Title"))


🧩 Internal field names:
ContentTypeId : Content Type ID
Title : Title
_ModerationComments : Approver Comments
File_x0020_Type : File Type
_ColorHex : Color
_ColorTag : Color Tag
_Emoji : Emoji
ComplianceAssetId : Compliance Asset Id
ContentSummary : ContentSummary
Benefits : Benefits
ContentOwner : ContentOwner
Function : Function
Site : Site
RecordId : RecordId
ID : ID
ContentType : Content Type
Modified : Modified
Created : Created
Author : Created By
Editor : Modified By
_HasCopyDestinations : Has Copy Destinations
_CopySource : Copy Source
owshiddenversion : owshiddenversion
WorkflowVersion : Workflow Version
_UIVersion : UI Version
_UIVersionString : Version
Attachments : Attachments
_ModerationStatus : Approval Status
Edit : Edit
LinkTitleNoMenu : Title
LinkTitle : Title
LinkTitle2 : Title
SelectTitle : Select
InstanceID : Instance ID
Order : Order
GUID : GUID
WorkflowInstanceID : Workflow Instance ID
FileRef : URL Path
FileDirRef : Path
Last_x0020_Modified : Modified
Created_x00

In [4]:
CERT_PEM

'./sharepoint_certificates/privkey.pem'